In [1]:
#part 1 Code
import pandas as pd 
import numpy as np 

filename = [
    '/Users/shishirporeddy/ECON433/weekly_patterns_2018_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2019_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2020_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2021_sample.csv.gz',
    '/Users/shishirporeddy/ECON433/weekly_patterns_2022_sample.csv.gz'
]
row_dictionary={}
core_qualification = 15 
jj_list = []

for file in filename:
    datax = pd.read_csv(file, compression='gzip', low_memory= False)
    row_dictionary[file] = len(datax)
    print(datax.columns)
    jj_data = datax[datax['brands'] =="Jimmy John's"]
    jj_list.append(jj_data)
    

concat_jj_data = pd.concat(jj_list, ignore_index=True)

import datetime
concat_jj_data['date_range_start'] = pd.to_datetime(concat_jj_data['date_range_start'], utc=True).dt.tz_localize(None)
concat_jj_data['date_range_end'] = pd.to_datetime(concat_jj_data['date_range_end'], utc=True).dt.tz_localize(None)

concat_jj_data[['dailyvisits1','dailyvisits2', 'dailyvisits3', 
                'dailyvisits4', 'dailyvisits5','dailyvisits6', 
                'dailyvisits7']] = concat_jj_data['visits_by_day'].str.strip("[]").str.split(",", expand=True).apply(pd.to_numeric, 
                                                                                                                     errors='coerce')

jj_long = concat_jj_data.melt(
    id_vars=['placekey', 'date_range_start', 'date_range_end', 'region', 'brands', 'city', 'raw_visit_counts', 'visits_by_day'],
    value_vars=['dailyvisits1','dailyvisits2', 'dailyvisits3', 'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7'],
    var_name='visits_column',
    value_name='dailyvisits')


jj_long["day_index"] = (
    jj_long["visits_column"].str.extract(r'(\d+)$') 
                   .astype(int)
)
jj_long["true_date"] = jj_long["date_range_start"] + pd.to_timedelta(jj_long["day_index"] - 1, unit="D")
jj_long["dayofweek"] = jj_long["true_date"].dt.day_name()
jj_long['dailyvisits'] = pd.to_numeric(jj_long['dailyvisits'], errors='coerce').fillna(0).astype(int)
jj_long['WKND'] = jj_long['dayofweek'].isin(['Saturday', 'Sunday'])
jj_long['threshold'] = np.where(jj_long['WKND'], 13, 18)
jj_long['manyvisits'] = np.where(jj_long['dailyvisits'] > jj_long['threshold'], 1, 0)

unique_in_region = (
    jj_long.groupby('region', as_index=False)['placekey'].nunique().rename(columns={'placekey':'number_of_stores'}))

jj_long = jj_long.merge(unique_in_region, on = 'region', how='left')
jj_long['core_biz_area'] = np.where(jj_long['number_of_stores'] >= core_qualification, 1, 0)

jj_long.to_csv("jimmy_johns_long_sample.csv", index=False)


print(jj_long.shape)
print(jj_long.head(5))
print("done")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#part 1 code
unique_days = jj_long["dayofweek"].unique()
print("Unique values in day_of_week:", unique_days)


weekend_total = (jj_long['WKND'] == 1).sum()
weekday_total = (jj_long['WKND'] == 0).sum()

print("weekend total: ", weekend_total)
print("weekday total: ", weekday_total)

Question 1:

The unique values of the dayofweek variable include Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, and Sunday.
The total observations for the weekend was 208,800 and for the weekday it was 522,000. 

In [ ]:
#part 2
#2(a) code

dailyvisits_summary = jj_long['dailyvisits'].describe()
dailyvisits_median = jj_long['dailyvisits'].median()
dailyvisits_mean = jj_long['dailyvisits'].mean()

print(dailyvisits_summary)
print("median: ", dailyvisits_median)
print("mean", dailyvisits_mean)

zero_dailyvisits = ( jj_long['dailyvisits'] == 0).sum()
hundred_dailyvisits = ( jj_long['dailyvisits'] >= 100).sum()
thousand_dailyvisits = ( jj_long['dailyvisits'] >= 1000).sum()

print("zero:", zero_dailyvisits)
print("above hundred:", hundred_dailyvisits)
print("above thousand:", thousand_dailyvisits)


In [ ]:
#2(b) code
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.histplot(data = jj_long[jj_long['dailyvisits'] <=100], x ='dailyvisits', bins = 101, edgecolor='blue')
plt.xlim(0,100)
plt.title("Dailyvisits Histogram:Sub-Range(0-100)")
plt.xlabel("Number of Dailyvisits")
plt.ylabel("count")

plt.show

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.histplot(data = jj_long, x ='dailyvisits', bins = 101, hue='WKND', multiple='dodge', edgecolors='red')
plt.xlim(0,100)
plt.title("Grouped Dailyvisits Histogram:Weekend vs Weekday")
plt.xlabel("Number of Dailyvisits")
plt.ylabel("count")
plt.legend(labels=['Weekend', 'Weekday'])
plt.show

#Question 2 

2a: 
There are a total of 730800 observations. 
The minimum value is 0.0, the max value is 5915, the median is 3.0, and the mean is 15.39. 
121,140 observation equal to 0. 
11229 observations equal to or greater than 100. 
1403  observations equal to or greater than 1000. 

2b: 
I chose the number of bins to be 101 because if I am choosing a range of 0-100 to reduce the skewnewss of the histogram, having 101 bins allows for each individual integer to have its own bin. 
I chose the sub-range of 100 in order to reduce the overall skewness shown by all the data.

2c: 
Weekdays have a larger proportion of data that falls under the very low dailyvisit range in comparison to Weekends. 

In [ ]:
#Part 3 
#3(a)

sum_per_day = (
    jj_long.groupby('dayofweek', as_index=False)['dailyvisits'].sum()
)
print(sum_per_day)

In [ ]:
#3(b)

sum_per_day['dailyvisits'] = pd.to_numeric(sum_per_day['dailyvisits'], errors='coerce')


plt.figure(figsize=(8,8))

plt.pie(
    x= sum_per_day['dailyvisits'],
    labels = sum_per_day['dayofweek'], 
    autopct='%1.1f%%'
)
plt.show()

#Question 3:

3(a): The results are shown in the table above. 

3(b): Pie chart depicting the total dailyvisits across the seven days of the week. 

In [ ]:
#Part 4 
#4(a)


sum_by_region = (
    jj_long.groupby('region', as_index=False)['dailyvisits'].mean()
)
print(sum_by_region)


In [ ]:
#4(b)

fig, ax = plt.subplots(figsize=(15,9), dpi=100)

barchart = ax.bar(
    x = sum_by_region['region'],
    height = sum_by_region['dailyvisits'], 
    color = 'red'
    
)
ax.set_xlabel("region")
ax.set_ylabel("mean daily visits")
ax.set_title("region based averages")
plt.tight_layout()

for bars in barchart:
    height= bars.get_height()
    ax.text(
        bars.get_x() + bars.get_width()/2, 
        height + 0.1, 
        f"{height:.1f}", 
        ha = "center" , 
        va = "bottom"
    )
    
plt.show()



In [ ]:
#4(c)

jj_long['covid'] = np.where(
    jj_long['true_date'] <= pd.to_datetime('2020-03-13'), 'Pre-Covid', 'Post-Covid')

region_covid = (
    jj_long.groupby(['region', 'covid'], as_index = False)['dailyvisits'].mean())

plt.figure(figsize=(15,9))
sns.barplot(
    data=region_covid, 
    x = 'region', 
    y = 'dailyvisits', 
    hue = 'covid', 
    palette = 'Dark2')

plt.title("Pre/Post Covid average's of daily visits by region")
plt.xlabel("Region")
plt.ylabel("Average Daily Visits")
plt.legend(title = "Covid Time")
plt.tight_layout()
plt.show()


#Question 4:

4(a): Results are shown in the table above, showing the average dailyvisits by region

4(b): Bar chart is visualized in order to show the dailyvisit averages based on the region. 

4(c): Bar chart is visualized in order to show pre and post covid-19 time period and how the daily averages by region is affected. One difference found between the two sets in the bar chart are that the most regions show a smaller daily average in post covid times in comparison to pre covid times. The second difference is that some regions have a larger drop off from pre covid times to post covid times in comparison to other regions where the drop off was a lot smaller.

In [ ]:
#part 5
#5(a)

jj_long['year_month'] = jj_long['true_date'].dt.to_period('M')

avg_by_month = (
    jj_long.groupby('year_month', as_index = False)['dailyvisits'].mean())
pd.set_option('display.max_rows', None)

print(avg_by_month)

In [ ]:
#5(b)
avg_by_month['start_year_month'] = avg_by_month['year_month'].dt.to_timestamp()

fig, ax = plt.subplots(figsize = (12,6))
ax.plot(
    avg_by_month['start_year_month'], 
    avg_by_month['dailyvisits'], 
    label = 'Avg Daily Visits')

vertical_line = avg_by_month['start_year_month'].dt.year.unique()
for year in vertical_line:
    new_year = pd.to_datetime(f'{year}-01-01')
    ax.axvline(new_year, color = 'black')
    
march_2020 = pd.to_datetime('2020-03-01')
ax.axvline(march_2020, color = 'red', label = 'March 2020')

ax.set_title("Avg dailyvisits by year-month")
ax.set_xlabel("Year-Month")
ax.set_ylabel("Avg dailyvisits")
ax.legend()
plt.show()


In [ ]:
#5(c)

jj_long['year_month'] = jj_long['true_date'].dt.to_period('M')

temp = (
    jj_long
    .groupby('year_month')['dailyvisits']
    .agg(['mean','median'])
    .reset_index()
)
temp.rename(columns={'mean':'dailyvisits_mean','median':'dailyvisits_median'}, inplace=True)

percentiles_25 = (
    jj_long
    .groupby('year_month')['dailyvisits']
    .apply(lambda x: np.percentile(x, 25))
    .reset_index(name='dailyvisits_25th')
)

percentiles_75 = (
    jj_long
    .groupby('year_month')['dailyvisits']
    .apply(lambda x: np.percentile(x, 75))
    .reset_index(name='dailyvisits_75th')
)

four_dailyvisits = (
    temp
    .merge(percentiles_25, on='year_month')
    .merge(percentiles_75, on='year_month')
)


four_dailyvisits['start_month'] =four_dailyvisits['year_month'].dt.to_timestamp()

fig, ax = plt.subplots(figsize=(12,6))

ax.plot(four_dailyvisits['start_month'], four_dailyvisits['dailyvisits_mean'], label = 'Mean', color='pink')
ax.plot(four_dailyvisits['start_month'], four_dailyvisits['dailyvisits_median'], label = 'Median', color='Green')
ax.plot(four_dailyvisits['start_month'], four_dailyvisits['dailyvisits_25th'], label = '25th Percentile', color='purple')
ax.plot(four_dailyvisits['start_month'], four_dailyvisits['dailyvisits_75th'], label = '75th Percentile', color='Blue')

ax.fill_between(
    four_dailyvisits['start_month'],
    four_dailyvisits['dailyvisits_25th'],
    four_dailyvisits['dailyvisits_75th'],
    color = 'gray', alpha = 0.2, label = '25th-75th Percentile Range')

    


vertical_line = avg_by_month['start_year_month'].dt.year.unique()
for year in vertical_line:
    new_year = pd.to_datetime(f'{year}-01-01')
    ax.axvline(new_year, color = 'black')
    
march_2020 = pd.to_datetime('2020-03-01')
ax.axvline(march_2020, color = 'red', label = 'March 2020')

ax.set_title("Dailyvisits by Months - (Mean, Median, 25th, 75th")
ax.set_xlabel("year-month")
ax.set_ylabel("Dailyvisits")
ax.legend()
plt.show()


Question 5:

5(a): Results shown in table 

5(b): Line chart displayed shows the average dailyvisits per year-month. 

5(c): Line chart for the 4 lines is displayed. The pattern for the mean is consitently higher than the 75th percentile, which would mean a right-skewed distribution. At the red line indidicating March 2020, the time covid happened, there was a drop in all measurables indicating the impact Covid-19 had.

In [ ]:
#Part 6: 

jj_long['year'] = jj_long['true_date'].dt.year


plt.figure(figsize=(12,6))
sns.boxplot(
    data = jj_long, 
    x='year',
    y='dailyvisits',
    color = 'blue'
)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Question 6: 

This box plot didn't resemble the normal box and whiskers type of plot. This box plot shows that the majority of the dailyvisits are at a low value, while the outliers are at higher values. 

In regards to the line plot from the previous question, this plot also exemplifies that there was a skewness in the data. 

In [ ]:
#Part 7 
import pandas as pd
import numpy as np
import geopandas as gpd
 
import matplotlib.pyplot as plt


us_state = (
    jj_long
    .groupby('region', as_index=False)['dailyvisits']
    .mean()
)

state_id = {
    'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05', 'CA': '06',
    'CO': '08', 'CT': '09', 'DE': '10', 'FL': '12', 'GA': '13',
    'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19',
    'KS': '20', 'KY': '21', 'LA': '22', 'ME': '23', 'MD': '24',
    'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28', 'MO': '29',
    'MT': '30', 'NE': '31', 'NV': '32', 'NH': '33', 'NJ': '34',
    'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39',
    'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44', 'SC': '45',
    'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 'VT': '50',
    'VA': '51', 'WA': '53', 'WV': '54', 'WI': '55', 'WY': '56',
    'DC': '11'
}

us_state['state_code'] = us_state['region'].map(state_id)



url_states = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
geoData = gpd.read_file(url_states)



combined_data = pd.merge(
    geoData, 
    us_state, 
    left_on ="id",
    right_on ="region", 
    how="left"
)

fig, ax = plt.subplots(figsize=(12,8))

combined_data.plot(
    column = 'dailyvisits',
    legend = True, 
    ax =ax)

ax.set_title("State by State dailyvisits")
ax.set_axis_off()
plt.show()


Question 7: 

(a) Based on a smell test based on public information, Nevada is the state with the highest average daily customers and according to the map the state of Nevada is highlighted yellow showing they have the highest average daily customers. 

(b)From this geographic visualization I learned that for some reason Nevada is obsessed with Jimmy John's. Most of the states range in the dark blue to purple range indidcating a lower average of daily customers. The states that are really purple could indicate the lack of presence of Jimmy John's in those state. 

(c)A business owner would look at this map and would see that the states that already have the ball rolling but not the highest averages could benfit from the implementation of an expansion of the business in those areas. Incorporating further marketing in these areas may drive their profits even higher as they are expanding their fan base(customers). 